# 第19章 利器

本章我们回过头来介绍一些之前被忽略的好东西。Python提供了一些虽非必要，但好用的功能---你不用这些功能仍然可以写出好代码---但是使用了这些技巧，有时可以编写更简洁，易读，高效的代码，甚至有时能同时兼顾这三个目标。

#### 19.1 条件表达式

我们在第5.4节见过条件语句。条件语句通常用来二选一；比如：

In [1]:
import math

x = 10
if x > 0:
    y = math.log(x)
else:
    y = float('nan')
# 这里判断x是否是正数。如果不是就会抛出异常。为了避免异常退出，设置特殊的浮点数，表示不是一个数字。
# 我们可用一个条件表达式来更简洁编写此语句：
y = math.log(x) if x > 0 else float('nan')
# 你可以像读英语一样读这行代码:"y gets log-x if x is greater than 0; otherwise it gets NaN.""

In [ ]:
# 递归函数有时候可以用条件表达式重写。比如，下面是一个阶乘函数的递归版本：
def factorial(n):
    if n ==0:
        return 1
    else:
        return n * factorial(n-1)
# 我们可以重写为：
def factorial(n):
    return 1 if n ==0 else n * factorial(n-1)

In [ ]:
# 条件表达式的另一个用途是处理可选参数。比如，下面是 GoodKangaroo中的init方法：
def __init__(self, name, contents=None):
    self.name = name
    if contents == None:
        contents = []
    self.pouch_contents = contents
# 我们可以这样改写：
def __init__(self, name, contents=None):
    self.name = name
    self.pouch_contents = [] if contents == None else contents
# 如果条件语句的两个分支都是简单表达式，要么是被返回，要么是被赋值给同一变量，那么你就可以用条件表达式来替换条件语句

文涛：原来这就是Python的利器，以前会把代码交给ChatGPT来修改，它写出这样的代码我还看不懂，现在才知道是怎么回事。学习真的是在节省时间，我只需要读这么一篇介绍就弄明白了它是什么，如果我去网上随便搜帖子或者自行摸索，不知道要花费多少时间呢。

#### 19.2 列表推导式

我们学习了map和filter模式。比如，下面函数接收一个字符串列表，将字符串方法 capitalize 映射到每个元素上，并返回一个新的字符串列表：

In [ ]:
def capitalize_all(t):
    res = []
    for s in t:
        res.append(s.capitalize())
    return res
# 我们可以用列表推导式来精简此函数：
def capitalize_all(t):
    return [s.capitalize() for s in t]
# 方括号操作符表示我们在构建一个新列表。方括号中的表达式，指定了列表中的元素，同时for语句声明了我们要遍历的序列。
# 列表推导式的语法略显奇怪，主要因为这个循环变量，如例子中的s，在定义之前，便出现在了表达式中。

# 列表推导式也可以用来过滤。比如，下面函数仅选择t中大写字母的元素，并返回新列表：
def only_upper(t):
    res = []
    for s in t:
        if s.isupper():
            res.append(s)
    return res
# 我们可以用列表推导式来重写：
def only_upper(t):
    return [s for s in t if s.isupper()]

列表推导式更加简洁，也更易读，至少对于简单表达式是这样的。而且，它们相比同样的for循环，要更快，甚至有时快很多。所以如果你因为我没有更早提及而生气，我理解。

但是，我要申辩一下，列表推导式通常更难调试，因为你不能在循环内部放置一个打印语句。我建议你只在计算足够简单的情况，也就是你上手便能正确完成的时候使用。对于新手来说，尽量别用。

#### 19.3 生成器表达式

生成器表达式和列表推导式相似，但它使用圆括号而不是方括号：

结果是一个生成器对象，同时其知晓如何遍历一个值序列。但是与列表推导式不同，它不会立即计算所有值，而是在被调用时才计算。

#### 19.4 any 和 all

Python提供了一个内置函数，any，其接收一个布尔值序列，如果任意一个值为True，则返回True。通常适用于列表：

In [ ]:
any([False, False, True]) # True

# 这个例子不太明显，因为和in运算符的效果一样。但我们可以用 any来重写第9.3节中，曾经写的一些搜索函数。
def avoids(word, forbidden):
    return not any(letter in forbidden for letter in word)

这个函数可以像读英语那样理解，"word avoids forbidden if there are not any forbidden letters in word." （如果单词中不包含任何禁用字母，那么单词便避免被禁用）

将 any 和生成器表达式一起使用，效率更高，因为只要遇到 True值，它便会立刻停止，所以不会计算整个序列。

Python还提供了另一个内置函数，all，如果序列中每个元素都是True，那么它便会返回True。

#### 19.5 集合（set）

在13.6节，我用字典来寻找出现在文档中，但是不在单词列表中的单词。Python提供了另一种内置类型，叫做 set，其效果很想字典中没有值的键的集合。向集合中添加元素是很快的；检查成员也很快。同时集合也提供了执行常见操作的方法和运算符。

In [ ]:
def subtract(d1, d2):
    res = dict()
    for key in d1:
        if key not in d2:
            res[key] = None
    return res
# 在这些字典中，值都是None，因为从没有使用它们。结果就是我们浪费了一些存储空间。
# 集合的差集操作有 difference方法，或者-操作符。所以我们可以将subtract像下面一样重写：
def subtract(d1, d2):
    return set(d1) - set(d2)
# 结果是一个集合，而不是字典，但对于像迭代这样的操作，其行为是相同的。

In [ ]:
# 本书的一些练习题可以用集合进行精简和优化。下面是习题10.7中，使用字典解决has_duplicates的一个方案：
def has_duplicates(t):
    d = {}
    for x in t:
        if x in d:
            return True
        d[x] = True
    return False
# 当一个元素首次出现，则被加入字典。如果同一元素再次出现，函数返回True。

# 使用集合，我们可以如下一般，实现相同的函数：
def has_duplicates(t):
    return len(set(t)) < len(t)

# 一个元素在集合中只能出现一次，所以如果t中有元素出现多次，那么集合长度必然小于t的长度。
# 如果没有重复的，集合大小和t的大小相同。
    
def uses_only(word, available):
    return set(word) <= set(available)
# 运算符<= 会检验一个集合是否是另一个的子集, 包括两者相等的情况, 
# 也就是如果 word 中的字母都出现在 available, 则返回真

#### 19.11 习题集

Exercise 19.1. 下面是个递归计算二项式系数的函数

In [6]:
def binomial_coeff(n, k):
    """Compute the binomial coefficient "n choose k".
    
    n: number of trials
    k: number of successes
    
    returns: int
    """
    if k ==0:
        return 1
    if n ==0:
        return 0
    
    res = binomial_coeff(n-1, k) + binomial_coeff(n-1, k-1)
    return res

# 使用嵌套条件表达式优化函数体。
result = binomial_coeff(6, 3)
result

20